In [210]:
import pandas as pd
import plotly.express as px
import os
import requests


In [211]:
local_project_path = r'C:\Users\antoi\\Documents\\Work&Learn\\JEDHA\\M03-DataCollection_Managment\\JEDHA-Projet-2-Kayak'
path = local_project_path+'\\my_booking\\my_booking\\city.csv'

df_city = pd.read_csv(path, header=None, names=['city','nan'])
cities = df_city['city'].str.replace(' ', '-').tolist()


api_url = 'https://nominatim.openstreetmap.org/search'

enriched_cities = []

for city in cities:
    city_req = city.replace(' ', '+')
#    print(city)
    payload = {'city' : city_req,
            'limit' : 1,
            'country' : 'france',
            'country-code' : 'fr',
            'format' : 'json'}
    
    info = requests.get(api_url, 
                          params=payload)
    
    enriched_cities.append({'city': city,
                            'lat_target_city' : float(info.json()[0]['lat']),
                            'lon_target_city' : float(info.json()[0]['lon'])})
    
df_city = pd.DataFrame.from_dict(enriched_cities)

In [212]:
local_project_path = r'C:\Users\antoi\\Documents\\Work&Learn\\JEDHA\\M03-DataCollection_Managment\\JEDHA-Projet-2-Kayak'
path = local_project_path+'\\my_booking\\my_booking'
print(path)
os.chdir(path)

df = pd.read_json('hotel.json')
display(df.head(20))

C:\Users\antoi\\Documents\\Work&Learn\\JEDHA\\M03-DataCollection_Managment\\JEDHA-Projet-2-Kayak\my_booking\my_booking


,city,lat,lon,name,review_score,url,description
0,Lille,50.636038,3.066937,Brueghel,7.9,https://www.booking.com/hotel/fr/brueghel.en-g...,This hotel is located in the heart of Lille. I...
1,Lille,50.636547,3.062342,Grand Hotel Bellevue - Grand Place,8.4,https://www.booking.com/hotel/fr/grandhotelbel...,"On the Grand Place in Old Lille, Grand Hotel B..."
2,Lille,50.637888,3.072685,Hotel Lille Europe,8.0,https://www.booking.com/hotel/fr/lille-europe....,Located just 250 metres from Lille-Flandres Tr...
3,Lille,50.646022,3.075871,Hotel Art Deco Euralille,7.6,https://www.booking.com/hotel/fr/hotelsrtdecor...,"Located 1 km from Lille-Europe Train Station, ..."
4,Lille,50.627831,3.063592,Moxy Lille City,8.5,https://www.booking.com/hotel/fr/moxy-lille-ci...,"Conveniently set in Lille, Moxy Lille City pro..."
5,Lille,50.634074,3.064229,OKKO Hotels Lille Centre,8.7,https://www.booking.com/hotel/fr/okko-hotels-l...,"OKKO Hotels Lille Centre has a fitness centre,..."
6,Lille,50.636246,3.069242,Hotel Chagnot,8.0,https://www.booking.com/hotel/fr/hotel-balladi...,Hotel Chagnot is situated in Lille city centre...
7,Lille,50.630759,3.058088,"Holiday Inn Express Lille Centre, an IHG Hotel",8.0,https://www.booking.com/hotel/fr/expressbyholi...,"Located in Lille city centre, this hotel is 35..."
8,Lille,50.620033,3.084567,Eklo Hotels Lille,7.4,https://www.booking.com/hotel/fr/eklo-hotels-l...,"Offering a terrace, Eklo Hotels Lille is set i..."
9,Lille,50.645698,3.072110,Campanile Lille Euralille,8.1,https://www.booking.com/hotel/fr/campanile-lil...,"Located 1 km from Lille Europe, 2 km from Lill..."


In [213]:
fig = px.scatter_mapbox(df,
                        lat='lat',
                        lon='lon',
                        color='review_score',
                        center=dict(lat=46.2, lon=2.2),
                        zoom=4,
                        color_continuous_scale=['white', 'yellow', 'red'],
                        mapbox_style='open-street-map',
                        hover_name='name',
                        hover_data=['city'],
                        # hover_data=['city', 'description2'],
                        height=600)

fig.show()

In [214]:
import plotly.graph_objects as go

# Define personalized colorscale
custom_colorscale = [
    [0, 'yellow'],
    [0.33, 'orange'],
    [1, 'red']
]

# Create a scattermapbox trace
trace = go.Scattermapbox(
    lat=df['lat'],
    lon=df['lon'],
    mode='markers',
    marker=dict(
        size=10,
        color=df['review_score'],
        colorscale=custom_colorscale,  # Use personalized colorscale
        cmin=df['review_score'].min(),
        cmax=df['review_score'].max(),
        colorbar=dict(title='Review Score')
    ),
    hoverinfo='text',
    text=df['name'] + '<br>' + df['city']+ df['description'],
)

# Define layout
layout = go.Layout(
    title='Map',
    mapbox=dict(
        style='carto-positron',
        center=dict(lat=46.2, lon=2.2),
        zoom=4
    ),
    height=600
)

# Create figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

In [215]:
df_city['rate'] = df_city.index 
df_city

,city,lat_target_city,lon_target_city,rate
0,Mont-Saint-Michel,48.635954,-1.511460,0
1,St-Malo,48.649518,-2.026041,1
2,Bayeux,49.276462,-0.702474,2
3,Le-Havre,49.493898,0.107973,3
4,Rouen,49.440459,1.093966,4
5,Paris,48.853495,2.348391,5
6,Amiens,49.894171,2.295695,6
7,Lille,50.636565,3.063528,7
8,Strasbourg,48.584614,7.750713,8
9,Chateau-du-Haut-Koenigsbourg,48.249523,7.345492,9


In [281]:
import pandas as pd
import plotly.graph_objs as go

city_df = pd.DataFrame({
    'City': df_city['city'],
    'Latitude': df_city['lat_target_city'],
    'Longitude': df_city['lon_target_city'],
    'Rate': df_city['rate']
})

hotel_df = pd.DataFrame({
    'Hotel': df['name'],
    'City': df['city'],
    'Latitude': df['lat'],
    'Longitude': df['lon'],
    'Rating' : df['review_score'],
    'Desc' : df['description']

})



In [282]:
import textwrap

# Suppose your DataFrame is named hotel_df
hotel_df['Wrapped_Desc'] = ['<br>'.join(textwrap.wrap(x, width=80)) for x in hotel_df['Desc']]


In [284]:
import pandas as pd
import plotly.graph_objs as go

top5_cities = ["Bayonne", "Collioure", "Ariege", "Biarritz", "Avignon"]

color_scale = [
    (0, 5, 'rgb(169,169,169)'),  
    (5, 7, 'rgb(255,255,0)'),    
    (7, 8, 'rgb(255,165,0)'),    
    (8, 9, 'rgb(255,0,0)'),      
    (9, 10, 'rgb(128,0,128)')    
]

fig = go.Figure()

for _, row in hotel_df.iterrows():
    rating = row['Rating']
    color = None
    for lower, upper, col in color_scale:
        if lower <= rating <= upper:
            color = col
            break
    # hover_text = f"Hotel: {row['Hotel']}<br>Rating: {rating}<br>Description:{row['Wrapped_Desc']}"
    hover_text = f"Hotel: {row['Hotel']}<br>Rating: {rating}"
    max_width = 0.1,  

    fig.add_trace(go.Scattergeo(
        lon=[row['Longitude']],
        lat=[row['Latitude']],
        mode='markers',
        marker=dict(size=17, symbol='circle', color=color, line=dict(color='black', width=1)),
        # name=row['Hotel'],
        name='',
        visible=False,
        # hoverinfo='text',
        # text=hover_text,
        hovertemplate=hover_text,
        # hovertemplate=hover_text
        hoverlabel=dict(
            bgcolor="skyblue",  
            bordercolor="black",
            font=dict(size=12), 
            align='left',  
    )
    ))

for _, row in city_df.iterrows():
    city_name = row['City']
    if city_name in top5_cities:
        color = 'gold'
    else:
        color = 'blue'


    fig.add_trace(go.Scattergeo(
        lon=[row['Longitude']],
        lat=[row['Latitude']],
        mode='markers',
        marker=dict(size=17, symbol='circle', color=color, line=dict(color='black', width=1)),
        name='',
        visible=True,
        hovertemplate=city_name
    ))

legend_annotations = [
    dict(
        x=0,
        y=0.8,
        xref='paper',
        yref='paper',
        xanchor='left',
        yanchor='middle',
        width=90,
        height=50,
        text='Top 5',
        bgcolor='gold',
        font=dict(color='white'),
        showarrow=False
    ),
    dict(
        x=0,
        y=0.8 - 0.14,
        xref='paper',
        yref='paper',
        xanchor='left',
        yanchor='middle',
        width=90,
        height=50,
        text='Other',
        bgcolor='blue',
        font=dict(color='white'),
        showarrow=False
    )
]

legend_annotations += [
    dict(
        x=0.9,
        y=0.8 - i * 0.14,
        xref='paper',
        yref='paper',
        xanchor='left',
        yanchor='middle',
        width=90,
        height=50,
        text=f'{lower}-{upper}',
        bgcolor=color,
        font=dict(color='white'),
        showarrow=False
    ) for i, (lower, upper, color) in enumerate(color_scale)
]

# Update layout with legend annotations
fig.update_layout(
    title='Hotels in Cities',
    showlegend=False,
    geo=dict(
        scope='europe',
        center=dict(lat=46, lon=2),  
        lonaxis_range=[-5, 10], 
        lataxis_range=[42, 52]
    ),
    height=600,
    width=1000,
    title_x=0.5,
    annotations=legend_annotations
)

# Add buttons for toggling between hotels and cities
fig.update_layout(
    updatemenus=[
        dict(
            type='buttons',
            buttons=[
                dict(label='Cities', method='update',
                     args=[{'visible': [False] * len(hotel_df) + [True] * len(city_df)}])
            ],
            x=0.1,  
            y=1,  
        ),
        dict(
            type='buttons',
            buttons=[
                dict(label='Hotels', method='update',
                     args=[{'visible': [True] * len(hotel_df) + [False] * len(city_df)}])
            ],
            x=1.025, 
            y=1, 
        )
    ]
)

fig.show()

In [218]:
import pandas as pd
import plotly.graph_objs as go

city_df = pd.DataFrame({
    'City': df_city['city'],
    'Latitude': df_city['lat_target_city'],
    'Longitude': df_city['lon_target_city'],
    'Rate': df_city['rate']
})

hotel_df = pd.DataFrame({
    'Hotel': df['name'],
    'City': df['city'],
    'Latitude': df['lat'],
    'Longitude': df['lon']
})

top5_cities = ["Bayonne", "Collioure", "Tarascon-sur-Ariège", "Biarritz", "Avignon"]


In [219]:
df.merge(df_city,on='city')

,city,lat,lon,name,review_score,url,description,lat_target_city,lon_target_city,rate
0,Lille,50.636038,3.066937,Brueghel,7.9,https://www.booking.com/hotel/fr/brueghel.en-g...,This hotel is located in the heart of Lille. I...,50.636565,3.063528,7
1,Lille,50.636547,3.062342,Grand Hotel Bellevue - Grand Place,8.4,https://www.booking.com/hotel/fr/grandhotelbel...,"On the Grand Place in Old Lille, Grand Hotel B...",50.636565,3.063528,7
2,Lille,50.637888,3.072685,Hotel Lille Europe,8.0,https://www.booking.com/hotel/fr/lille-europe....,Located just 250 metres from Lille-Flandres Tr...,50.636565,3.063528,7
3,Lille,50.646022,3.075871,Hotel Art Deco Euralille,7.6,https://www.booking.com/hotel/fr/hotelsrtdecor...,"Located 1 km from Lille-Europe Train Station, ...",50.636565,3.063528,7
4,Lille,50.627831,3.063592,Moxy Lille City,8.5,https://www.booking.com/hotel/fr/moxy-lille-ci...,"Conveniently set in Lille, Moxy Lille City pro...",50.636565,3.063528,7
...,...,...,...,...,...,...,...,...,...,...
870,La-Rochelle,46.157796,-1.138319,Hôtel Kyriad La Rochelle Centre Ville,8.5,https://www.booking.com/hotel/fr/kyriad-la-roc...,Situated just 1 km from the old harbour and th...,46.159113,-1.152043,34
871,La-Rochelle,46.162705,-1.160611,Le Manoir Hôtel,8.6,https://www.booking.com/hotel/fr/le-manoir-la-...,"Set in a 19th-century building in La Rochelle,...",46.159113,-1.152043,34
872,La-Rochelle,46.154502,-1.144672,B&B HOTEL La Rochelle Centre,8.1,https://www.booking.com/hotel/fr/b-amp-b-la-ro...,Set in the La Rochelle City Centre district in...,46.159113,-1.152043,34
873,La-Rochelle,46.147176,-1.152722,B&B HOTEL La Rochelle Centre Les Minimes,7.9,https://www.booking.com/hotel/fr/altica-la-roc...,"""NEW ROOMS!!!""<br><br>The team of the B&B HOTE...",46.159113,-1.152043,34


In [220]:
import pandas as pd
import math

def haversine(lat1, lon1, lat2, lon2):
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    R = 6371.0
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    
    return distance

In [221]:
df['distance'] = df.merge(df_city,on='city').apply(lambda row: haversine(row['lat'], row['lon'], row['lat_target_city'], row['lon_target_city']), axis=1)

In [222]:
df.sort_values('distance')
df.sort_values('review_score')

,city,lat,lon,name,review_score,url,description,distance
46,Le-Havre,49.492303,0.122933,Hôtel Le Green,2.8,https://www.booking.com/hotel/fr/le-green.en-g...,"Located in the centre of Le Havre, this hotel ...",1.094919
273,Besancon,47.218537,5.982122,hotelF1 Besancon Ouest Micropolis,4.2,https://www.booking.com/hotel/fr/hotelf1-besan...,Hotelf1 Besancon Ouest Micropolis - offers acc...,0.303225
788,Bayonne,43.489623,-1.451075,hotelF1 Bayonne,5.4,https://www.booking.com/hotel/fr/hotelf1-bayon...,HotelF1 Bayonne offers accommodation a 7-minut...,1.901761
660,Montauban,44.007347,1.385129,hotelF1 Montauban,5.4,https://www.booking.com/hotel/fr/hotelf1-monta...,"hotelF1 Montauban is situated in Montauban, 45...",2.664818
781,Bayonne,43.490490,-1.452035,Premiere Classe Bayonne,5.6,https://www.booking.com/hotel/fr/premiere-clas...,"Premiere Classe Bayonne, easily accessible fro...",1.801359
...,...,...,...,...,...,...,...,...
64,Bayeux,49.249366,-0.712469,Château Saint Gilles,9.6,https://www.booking.com/hotel/fr/chateau-saint...,"Situated in Bayeux, 4.6 km from Baron Gerard M...",3.099051
667,Montauban,44.113582,1.266295,L'Ostalas,10.0,https://www.booking.com/hotel/fr/l-39-ostalas....,"Set in Lafrançaise, 10 km from Les Aiguillons ...",12.813119
835,Collioure,42.528883,3.079394,Maison Nova - Boutique Hôtel & Spa,10.0,https://www.booking.com/hotel/fr/boutique-mais...,"Set in Collioure, 600 metres from Boramar Beac...",0.525978
663,Montauban,44.020996,1.349240,Dali Hôtel Montauban,NaN,https://www.booking.com/hotel/fr/dali-montauba...,"Located in Montauban, Dali Hôtel Montauban fea...",0.596665
